In [1]:
from Bio import Phylo
from Bio.Phylo.TreeConstruction import DistanceMatrix, DistanceTreeConstructor
from iterpop import iterpop as ip
from keyname import keyname as kn
from matplotlib import colors as mpl_colors
from teeplot import teeplot as tp

import Bio
import csv
import glob
import itertools
import json
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import networkx as nx
import numpy as np
import pandas as pd
import scipy as sci
import seaborn as sns
import urllib.request as request

from pathlib import Path

import json

from etlib.PlotTools.PlotTools import enable_pretty_graphing
enable_pretty_graphing()

# get data

In [2]:
filenames = glob.glob("dump/stint=*/*/a=genome+criteria=abundance+*thread=0*.json")

In [3]:
# todo: store this as a tar gz and upload it to osf !!!!!
# then decompress inside temp directory
# also push all data and graphs to github

# preprocess data

In [4]:
stint_tags = {}
tag_nums = set()  # for logging
for filename in filenames:
    file_data = kn.unpack(filename)
    with open(filename, 'r') as file:
        data = json.load(file)
        data = data['value0']['event_tags']['tags']
        tags = [v['value0']['value0'] for k, v in data.items()]
        tag_nums.add(len(tags))
        stint_tags[file_data['stint']] = tags
stint_tags = {int(k): v for k, v in stint_tags.items()}
stint_tags = dict(sorted(stint_tags.items()))
del stint_tags[101]
del stint_tags[102]

print("num tags", ip.popsingleton(tag_nums))


num tags 35


# generate distance matrix

## we're using hamming distance

In [5]:
def hamming_distance(a, b):
    return (a ^ b).bit_count()

## calculate all pairwise distances and store them in the matrix

In [6]:
matrix = np.zeros((len(stint_tags), len(stint_tags)))
pairwise = itertools.combinations(list(stint_tags.keys()), 2)

for a, b in pairwise:
    matrix[a][b] = sum(hamming_distance(x, y) for x, y in zip(stint_tags[a], stint_tags[b]))

# store data for later use

In [7]:
# make dir if it doesn't already exist
Path("data/").mkdir(exist_ok=True)
# save distance matrix as json
np.savetxt("data/distance_matrix.matrix", matrix) # store as txt with save_txt
# save stint tags
with open("data/stint_tags.json", 'w') as f:
    f.write(json.dumps(stint_tags, sort_keys=True))

In [8]:
matrix

array([[   0.,  150., 1116., ..., 1110., 1106., 1108.],
       [   0.,    0., 1124., ..., 1104., 1100., 1104.],
       [   0.,    0.,    0., ...,  488.,  490.,  488.],
       ...,
       [   0.,    0.,    0., ...,    0.,   12.,   10.],
       [   0.,    0.,    0., ...,    0.,    0.,   10.],
       [   0.,    0.,    0., ...,    0.,    0.,    0.]])